The part that connect to the mailbox, download them and add a unique ID to them:

In [ ]:
import os.path
import base64
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
import uuid

SCOPES = ['https://www.googleapis.com/auth/gmail.readonly', 'https://www.googleapis.com/auth/gmail.modify']


def returnEmails(): #use credentials to connect to the mailbox
    creds = None
    if os.path.exists('codes/token.json'):
        creds = Credentials.from_authorized_user_file('codes/token.json', SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('codes/my_cred_file.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('codes/token.json', 'w') as token:
            token.write(creds.to_json())

    email_data_list = []  # Initialiser une liste pour stocker les informations des emails

    try:
        service = build('gmail', 'v1', credentials=creds)
        results = service.users().messages().list(userId='me', labelIds=['INBOX'], q="is:unread").execute()
        messages = results.get('messages', [])
        
        if not messages:
            print('No new messages.')
        else:
            for message in messages:
                msg = service.users().messages().get(userId='me', id=message['id']).execute()
                
                # Extract email headers for subject, sender, and date
                email_data = msg['payload']['headers']
                subject = None
                from_name = None
                date = None
                for header in email_data:
                    if header['name'] == 'Subject':
                        subject = header['value']
                    if header['name'] == 'From':
                        from_name = header['value']
                    if header['name'] == 'Date':
                        date = header['value']
                
                # Extract the plain text body
                body = ""
                if 'parts' in msg['payload']:
                    for part in msg['payload']['parts']:
                        if part['mimeType'] == 'text/plain':
                            data = part['body']["data"]
                            byte_code = base64.urlsafe_b64decode(data)
                            body += byte_code.decode("utf-8")
                
                # Ajouter les informations dans un dictionnaire
                email_info = {
                    "email_id": str(uuid.uuid4()),
                    "from": from_name,
                    "subject": subject,
                    "date": date,
                    "body": body
                }
                
                # Ajouter ce dictionnaire à la liste
                email_data_list.append(email_info)
                
                # Mark the message as read
                service.users().messages().modify(userId='me', id=message['id'], body={'removeLabelIds': ['UNREAD']}).execute()
    
    except Exception as error:
        print(f'An error occurred: {error}')
    
    # Retourner la liste des emails
    return email_data_list

# Utilisation de la fonction
#emails = returnEmails()


The part wash the text and insert it in a json file:

In [ ]:
import os
import json
import re

def clean_all_urls(text):
    # Suppression des sauts de ligne et autres caractères d'échappement
    text = text.replace('\n', ' ').replace('\r', ' ').replace('\t', ' ')
    
    # Suppression des espaces multiples par un seul espace
    text = re.sub(r'\s+', ' ', text).strip()
    
    # Regex pour capturer toutes les URLs dans le texte
    pattern = r"(https?://[^\s]+|www\.[^\s]+|mailto:[^\s]+)"
    
    # Remplacer toutes les URLs par leur forme texte (non cliquable)
    text = re.sub(pattern, r'\g<0>', text)  # Remplace par le texte de l'URL lui-même
    
    return text.strip()

def remove_emojis(text):
    # Pattern Unicode pour détecter les emojis
    emoji_pattern = re.compile(
        "["
        "\U0001F600-\U0001F64F"  # emoticônes
        "\U0001F300-\U0001F5FF"  # symboles et pictogrammes divers
        "\U0001F680-\U0001F6FF"  # transport et symboles
        "\U0001F700-\U0001F77F"  # symboles alchimiques
        "\U0001F780-\U0001F7FF"  # symboles géométriques
        "\U0001F800-\U0001F8FF"  # symboles divers et pictogrammes
        "\U0001F900-\U0001F9FF"  # émoticônes supplémentaires et autres
        "\U0001FA00-\U0001FA6F"  # objets divers
        "\U00002702-\U000027B0"  # divers
        "\U000024C2-\U0001F251"
        "]+", flags=re.UNICODE)
    
    return emoji_pattern.sub(r'', text)

def save_email_to_file(email_data, filename="codes/mails.json"):
    # Charger les emails existants s'ils existent avec un encodage utf-8
    if os.path.exists(filename):
        try:
            with open(filename, 'r', encoding='utf-8') as file:
                existing_data = json.load(file)
                email_list = existing_data.get("emails", [])  # Récupérer la liste des emails
        except UnicodeDecodeError:
            # Si l'encodage utf-8 échoue, tenter avec un autre encodage
            with open(filename, 'r', encoding='cp1252', errors='ignore') as file:
                existing_data = json.load(file)
                email_list = existing_data.get("emails", [])  # Récupérer la liste des emails
    else:
        email_list = []

    # Appliquer le nettoyage des URLs, emojis, et autres caractères à chaque email
    for email in email_data:
        if "body" in email:
            email["body"] = clean_all_urls(email["body"])
            email["body"] = remove_emojis(email["body"])

    # Ajouter les nouveaux emails nettoyés à la liste
    email_list.extend(email_data)

    # Créer le dictionnaire à enregistrer
    data_to_save = {
        "emails": email_list
    }

    # Enregistrer dans le fichier JSON avec un encodage utf-8
    with open(filename, 'w', encoding='utf-8') as file:
        json.dump(data_to_save, file, ensure_ascii=False, indent=4)

    print(f"{len(email_data)} emails saved to {filename}")

the part that choose the right mistral agent:

In [ ]:
from mistralai import Mistral


def agentclassification(query):
    api_key = api_key
    
    client = Mistral(api_key=api_key)

    response = client.agents.complete(
                agent_id= agent_id,
                messages = [
                    {
                        "role": "user",
                        "content": query
                    },
                ]
            )
    result = response.choices[0].message.content
    return print(result)

The part the use the agent to produce a recognition:

In [ ]:
from mistralai import Mistral


def agentclassification(query):
    api_key = "S1pZGonHlgCIFMUzm3Y6O5jShDbJegMl"
    
    client = Mistral(api_key=api_key)

    response = client.agents.complete(
                agent_id= "ag:becb4bf0:20241012:redirection:8e1c095d",
                messages = [
                    {
                        "role": "user",
                        "content": query
                    },
                ]
            )
    result = response.choices[0].message.content
    return print(result)


The main.py that lunch the process:

In [ ]:
#from connexionboitemail import readEmails
#from connexionboitemailparts import readEmailsPart
from returnmails import returnEmails
from savemailstojson import save_email_to_file
#readEmails()
#readEmailsPart()


##Run on my env python12

emails = returnEmails()

if emails:
    save_email_to_file(emails, "codes/mails.json")
    


What my personnal mails look like after the process:

In [ ]:
{
    "emails": [
        {
            "email_id": "33e66ba3-c513-4f86-a808-71a031278536",
            "from": "Alertes LinkedIn Jobs <jobalerts-noreply@linkedin.com>",
            "subject": "“Développeur Python” : ALTEN - Développeur Python, C# - Finance de marché (F/H) et plus encore",
            "date": "Wed, 15 May 2024 14:37:27 +0000 (UTC)",
            "body": "Votre alerte Emploi pour Développeur Python (Issy-les-Moulineaux) 3 nouvelles offres d’emploi correspondent à vos préférences. Développeur Python, C# - Finance de marché (F/H) ALTEN Île-de-France, France Cette entreprise recrute activement Postulez avec un CV et un profil Voir l’offre d’emploi : https://www.linkedin.com/comm/jobs/view/3911737204/?trackingId=JijiDZuc%2F7xPcAHi1ENtGg%3D%3D&refId=ByteString%28length%3D16%2Cbytes%3Dcf4af818...bf992398%29&lipi=urn%3Ali%3Apage%3Aemail_email_job_alert_digest_01%3BNuLCBucGS4SNfZf5gmjQJw%3D%3D&midToken=AQErSc3Sdxmcdw&midSig=2oakGHYYuJmbg1&trk=eml-email_job_alert_digest_01-job_card-0-view_job&trkEmail=eml-email_job_alert_digest_01-job_card-0-view_job-null-bwd1d4~lw7wbvr2~al-null-null&eid=bwd1d4-lw7wbvr2-al&otpToken=MTUwNDE3ZTQxYTJkYzBjMGIwMjQwNGVkNDcxYmUwYmM4Y2M2ZDQ0NzljYWM4OTYxNzljZjAzNmE0YzVkNTVmMmY3ZDFkZmViNzJiMWU2ZmI1MTlkZjAyNWEyYjExY2NmYjFlYjVkOTYxYTFhMjIxZDQzZjU0OSwxLDE%3D --------------------------------------------------------- Developpeur Python systeme Linux H/F Jooble Arpajon Cette entreprise recrute activement Voir l’offre d’emploi : https://www.linkedin.com/comm/jobs/view/3926508232/?trackingId=LsaziUhW5RE%2BbQE9jcm%2FxA%3D%3D&refId=ByteString%28length%3D16%2Cbytes%3Dcf4af818...bf992398%29&lipi=urn%3Ali%3Apage%3Aemail_email_job_alert_digest_01%3BNuLCBucGS4SNfZf5gmjQJw%3D%3D&midToken=AQErSc3Sdxmcdw&midSig=2oakGHYYuJmbg1&trk=eml-email_job_alert_digest_01-job_card-0-view_job&trkEmail=eml-email_job_alert_digest_01-job_card-0-view_job-null-bwd1d4~lw7wbvr2~al-null-null&eid=bwd1d4-lw7wbvr2-al&otpToken=MTUwNDE3ZTQxYTJkYzBjMGIwMjQwNGVkNDcxYmUwYmM4Y2M2ZDQ0NzljYWM4OTYxNzljZjAzNmE0YzVkNTVmMmY3ZDFkZmViNzJiMWU2ZmI1MTlkZjAyNWEyYjExY2NmYjFlYjVkOTYxYTFhMjIxZDQzZjU0OSwxLDE%3D --------------------------------------------------------- Développeur Python/React (h/f) Free-Work (ex Freelance-info Carriere-info) Paris et périphérie Voir l’offre d’emploi : https://www.linkedin.com/comm/jobs/view/3925657009/?trackingId=e1sRRlB3VFvziMHkuJfD7g%3D%3D&refId=ByteString%28length%3D16%2Cbytes%3Dcf4af818...bf992398%29&lipi=urn%3Ali%3Apage%3Aemail_email_job_alert_digest_01%3BNuLCBucGS4SNfZf5gmjQJw%3D%3D&midToken=AQErSc3Sdxmcdw&midSig=2oakGHYYuJmbg1&trk=eml-email_job_alert_digest_01-job_card-0-view_job&trkEmail=eml-email_job_alert_digest_01-job_card-0-view_job-null-bwd1d4~lw7wbvr2~al-null-null&eid=bwd1d4-lw7wbvr2-al&otpToken=MTUwNDE3ZTQxYTJkYzBjMGIwMjQwNGVkNDcxYmUwYmM4Y2M2ZDQ0NzljYWM4OTYxNzljZjAzNmE0YzVkNTVmMmY3ZDFkZmViNzJiMWU2ZmI1MTlkZjAyNWEyYjExY2NmYjFlYjVkOTYxYTFhMjIxZDQzZjU0OSwxLDE%3D --------------------------------------------------------- Voir toutes les offres d’emploi sur LinkedIn : https://www.linkedin.com/comm/jobs/search?geoId=105839861&f_TPR=a1715695628-&savedSearchId=1739556099&origin=JOB_ALERT_EMAIL&lipi=urn%3Ali%3Apage%3Aemail_email_job_alert_digest_01%3BNuLCBucGS4SNfZf5gmjQJw%3D%3D&midToken=AQErSc3Sdxmcdw&midSig=2oakGHYYuJmbg1&trk=eml-email_job_alert_digest_01-job~alert-0-see~all~jobs~text&trkEmail=eml-email_job_alert_digest_01-job~alert-0-see~all~jobs~text-null-bwd1d4~lw7wbvr2~al-null-null&eid=bwd1d4-lw7wbvr2-al&otpToken=MTUwNDE3ZTQxYTJkYzBjMGIwMjQwNGVkNDcxYmUwYmM4Y2M2ZDQ0NzljYWM4OTYxNzljZjAzNmE0YzVkNTVmMmY3ZDFkZmViNzJiMWU2ZmI1MTlkZjAyNWEyYjExY2NmYjFlYjVkOTYxYTFhMjIxZDQzZjU0OSwxLDE%3D Découvrez les offres d’emploi pour lesquelles vous êtes l’un des meilleurs candidats https://www.linkedin.com/comm/premium/products/?utype=job&lipi=urn%3Ali%3Apage%3Aemail_email_job_alert_digest_01%3BNuLCBucGS4SNfZf5gmjQJw%3D%3D&midToken=AQErSc3Sdxmcdw&midSig=2oakGHYYuJmbg1&trk=eml-email_job_alert_digest_01-job~alert-0-premium~upsell~text&trkEmail=eml-email_job_alert_digest_01-job~alert-0-premium~upsell~text-null-bwd1d4~lw7wbvr2~al-null-null&eid=bwd1d4-lw7wbvr2-al&otpToken=MTUwNDE3ZTQxYTJkYzBjMGIwMjQwNGVkNDcxYmUwYmM4Y2M2ZDQ0NzljYWM4OTYxNzljZjAzNmE0YzVkNTVmMmY3ZDFkZmViNzJiMWU2ZmI1MTlkZjAyNWEyYjExY2NmYjFlYjVkOTYxYTFhMjIxZDQzZjU0OSwxLDE%3D ---------------------------------------- Cet e-mail est destiné à Thibault Goutorbe (Développeur Python/R, spécialisé en Data/IA, à la recherche d’un emploi.) Découvrez pourquoi nous précisons ceci : https://www.linkedin.com/help/linkedin/answer/4788?lang=fr&lipi=urn%3Ali%3Apage%3Aemail_email_job_alert_digest_01%3BNuLCBucGS4SNfZf5gmjQJw%3D%3D&midToken=AQErSc3Sdxmcdw&midSig=2oakGHYYuJmbg1&trk=eml-email_job_alert_digest_01-SecurityHelp-0-textfooterglimmer&trkEmail=eml-email_job_alert_digest_01-SecurityHelp-0-textfooterglimmer-null-bwd1d4~lw7wbvr2~al-null-null&eid=bwd1d4-lw7wbvr2-al&otpToken=MTUwNDE3ZTQxYTJkYzBjMGIwMjQwNGVkNDcxYmUwYmM4Y2M2ZDQ0NzljYWM4OTYxNzljZjAzNmE0YzVkNTVmMmY3ZDFkZmViNzJiMWU2ZmI1MTlkZjAyNWEyYjExY2NmYjFlYjVkOTYxYTFhMjIxZDQzZjU0OSwxLDE%3D Vous recevez des e-mails d’alertes Emploi. Gérez vos alertes Emploi : https://www.linkedin.com/comm/jobs/alerts?lipi=urn%3Ali%3Apage%3Aemail_email_job_alert_digest_01%3BNuLCBucGS4SNfZf5gmjQJw%3D%3D&midToken=AQErSc3Sdxmcdw&midSig=2oakGHYYuJmbg1&trk=eml-email_job_alert_digest_01-null-0-null&trkEmail=eml-email_job_alert_digest_01-null-0-null-null-bwd1d4~lw7wbvr2~al-null-null&eid=bwd1d4-lw7wbvr2-al&otpToken=MTUwNDE3ZTQxYTJkYzBjMGIwMjQwNGVkNDcxYmUwYmM4Y2M2ZDQ0NzljYWM4OTYxNzljZjAzNmE0YzVkNTVmMmY3ZDFkZmViNzJiMWU2ZmI1MTlkZjAyNWEyYjExY2NmYjFlYjVkOTYxYTFhMjIxZDQzZjU0OSwxLDE%3D Se désabonner : https://www.linkedin.com/job-alert-email-unsubscribe?savedSearchId=1739556099&lipi=urn%3Ali%3Apage%3Aemail_email_job_alert_digest_01%3BNuLCBucGS4SNfZf5gmjQJw%3D%3D&midToken=AQErSc3Sdxmcdw&midSig=2oakGHYYuJmbg1&ek=email_job_alert_digest_01&e=bwd1d4-lw7wbvr2-al&eid=bwd1d4-lw7wbvr2-al&m=unsubscribe&ts=footerGlimmer&li=0&t=plh · Aide : https://www.linkedin.com/help/linkedin/answer/67?lang=fr&lipi=urn%3Ali%3Apage%3Aemail_email_job_alert_digest_01%3BNuLCBucGS4SNfZf5gmjQJw%3D%3D&midToken=AQErSc3Sdxmcdw&midSig=2oakGHYYuJmbg1&trk=eml-email_job_alert_digest_01-help-0-textfooterglimmer&trkEmail=eml-email_job_alert_digest_01-help-0-textfooterglimmer-null-bwd1d4~lw7wbvr2~al-null-null&eid=bwd1d4-lw7wbvr2-al&otpToken=MTUwNDE3ZTQxYTJkYzBjMGIwMjQwNGVkNDcxYmUwYmM4Y2M2ZDQ0NzljYWM4OTYxNzljZjAzNmE0YzVkNTVmMmY3ZDFkZmViNzJiMWU2ZmI1MTlkZjAyNWEyYjExY2NmYjFlYjVkOTYxYTFhMjIxZDQzZjU0OSwxLDE%3D © 2024 LinkedIn Ireland Unlimited Company, Wilton Plaza, Wilton Place, Dublin 2.LinkedIn est le nom commercial déposé de LinkedIn Ireland Unlimited Company. LinkedIn et le logo de LinkedIn sont des marques déposées de LinkedIn."
        },
        {
            "email_id": "7f92d1f3-d0de-4628-b016-b4e6a164b991",
            "from": "Alertes LinkedIn Jobs <jobalerts-noreply@linkedin.com>",
            "subject": "“Développeur Python” : Klanik - Développeur Python/Développeuse Python et plus encore",
            "date": "Tue, 14 May 2024 20:40:20 +0000 (UTC)",
            "body": "Votre alerte Emploi pour Développeur Python (Issy-les-Moulineaux) 11 nouvelles offres d’emploi correspondent à vos préférences. Développeur Python/Développeuse Python Klanik Paris et périphérie Cette entreprise recrute activement Postulez avec un CV et un profil Voir l’offre d’emploi : https://www.linkedin.com/comm/jobs/view/3925214707/?trackingId=M0iqki%2B1IRpeCxBtmukZ0Q%3D%3D&refId=ByteString%28length%3D16%2Cbytes%3Dee2a5148...a4daa673%29&lipi=urn%3Ali%3Apage%3Aemail_email_job_alert_digest_01%3Bvtw3tFEUQ%2BScN%2BCR5Nu6yg%3D%3D&midToken=AQErSc3Sdxmcdw&midSig=155djI3kRUlbg1&trk=eml-email_job_alert_digest_01-job_card-0-view_job&trkEmail=eml-email_job_alert_digest_01-job_card-0-view_job-null-bwd1d4~lw6uuuqc~7b-null-null&eid=bwd1d4-lw6uuuqc-7b&otpToken=MTUwNDE3ZTQxYTJkYzBjMGIwMjQwNGVkNDcxYmUwYjU4NmNjZDE0ODlhYTk4ZjYxNzljZjAzNmE0YzVkNTVmMmY3ZDFkZmUxMTZkMmYzZTg2YzlkZjYzZDhlZmEyOTgxNTk0MDZiMjZiYjliNjUxZTc0NjMyNywxLDE%3D --------------------------------------------------------- Développeur Python/Développeuse Python (H/F) XRAYS TRADING Ville de Paris Cette entreprise recrute activement Postulez avec un CV et un profil Voir l’offre d’emploi : https://www.linkedin.com/comm/jobs/view/3850292973/?trackingId=VSXY3tth%2BOlJynRL1dYJ4w%3D%3D&refId=ByteString%28length%3D16%2Cbytes%3Dee2a5148...a4daa673%29&lipi=urn%3Ali%3Apage%3Aemail_email_job_alert_digest_01%3Bvtw3tFEUQ%2BScN%2BCR5Nu6yg%3D%3D&midToken=AQErSc3Sdxmcdw&midSig=155djI3kRUlbg1&trk=eml-email_job_alert_digest_01-job_card-0-view_job&trkEmail=eml-email_job_alert_digest_01-job_card-0-view_job-null-bwd1d4~lw6uuuqc~7b-null-null&eid=bwd1d4-lw6uuuqc-7b&otpToken=MTUwNDE3ZTQxYTJkYzBjMGIwMjQwNGVkNDcxYmUwYjU4NmNjZDE0ODlhYTk4ZjYxNzljZjAzNmE0YzVkNTVmMmY3ZDFkZmUxMTZkMmYzZTg2YzlkZjYzZDhlZmEyOTgxNTk0MDZiMjZiYjliNjUxZTc0NjMyNywxLDE%3D --------------------------------------------------------- Développeur Python H/F DAVRICOURT | Certified Positive Company® Paris Cette entreprise recrute activement Voir l’offre d’emploi : https://www.linkedin.com/comm/jobs/view/3925449098/?trackingId=YYKXHVgFIT58pXTnyjsoXQ%3D%3D&refId=ByteString%28length%3D16%2Cbytes%3Dee2a5148...a4daa673%29&lipi=urn%3Ali%3Apage%3Aemail_email_job_alert_digest_01%3Bvtw3tFEUQ%2BScN%2BCR5Nu6yg%3D%3D&midToken=AQErSc3Sdxmcdw&midSig=155djI3kRUlbg1&trk=eml-email_job_alert_digest_01-job_card-0-view_job&trkEmail=eml-email_job_alert_digest_01-job_card-0-view_job-null-bwd1d4~lw6uuuqc~7b-null-null&eid=bwd1d4-lw6uuuqc-7b&otpToken=MTUwNDE3ZTQxYTJkYzBjMGIwMjQwNGVkNDcxYmUwYjU4NmNjZDE0ODlhYTk4ZjYxNzljZjAzNmE0YzVkNTVmMmY3ZDFkZmUxMTZkMmYzZTg2YzlkZjYzZDhlZmEyOTgxNTk0MDZiMjZiYjliNjUxZTc0NjMyNywxLDE%3D --------------------------------------------------------- Développeur.se Python/Django “Full stack” - Contrat d'apprentissage Interlogement93 Montreuil Voir l’offre d’emploi : https://www.linkedin.com/comm/jobs/view/3924978389/?trackingId=mta7AiIqS8Rlje9XajxKHw%3D%3D&refId=ByteString%28length%3D16%2Cbytes%3Dee2a5148...a4daa673%29&lipi=urn%3Ali%3Apage%3Aemail_email_job_alert_digest_01%3Bvtw3tFEUQ%2BScN%2BCR5Nu6yg%3D%3D&midToken=AQErSc3Sdxmcdw&midSig=155djI3kRUlbg1&trk=eml-email_job_alert_digest_01-job_card-0-view_job&trkEmail=eml-email_job_alert_digest_01-job_card-0-view_job-null-bwd1d4~lw6uuuqc~7b-null-null&eid=bwd1d4-lw6uuuqc-7b&otpToken=MTUwNDE3ZTQxYTJkYzBjMGIwMjQwNGVkNDcxYmUwYjU4NmNjZDE0ODlhYTk4ZjYxNzljZjAzNmE0YzVkNTVmMmY3ZDFkZmUxMTZkMmYzZTg2YzlkZjYzZDhlZmEyOTgxNTk0MDZiMjZiYjliNjUxZTc0NjMyNywxLDE%3D --------------------------------------------------------- Développeur Python – Poste en client final (Finance de marchés) – CDI – Paris – 120/180K€ MR SEARCH Ville de Paris Cette entreprise recrute activement Postulez avec un CV et un profil Voir l’offre d’emploi : https://www.linkedin.com/comm/jobs/view/3863595888/?trackingId=NteZ3YLuLRwOfMn2UnZDZg%3D%3D&refId=ByteString%28length%3D16%2Cbytes%3Dee2a5148...a4daa673%29&lipi=urn%3Ali%3Apage%3Aemail_email_job_alert_digest_01%3Bvtw3tFEUQ%2BScN%2BCR5Nu6yg%3D%3D&midToken=AQErSc3Sdxmcdw&midSig=155djI3kRUlbg1&trk=eml-email_job_alert_digest_01-job_card-0-view_job&trkEmail=eml-email_job_alert_digest_01-job_card-0-view_job-null-bwd1d4~lw6uuuqc~7b-null-null&eid=bwd1d4-lw6uuuqc-7b&otpToken=MTUwNDE3ZTQxYTJkYzBjMGIwMjQwNGVkNDcxYmUwYjU4NmNjZDE0ODlhYTk4ZjYxNzljZjAzNmE0YzVkNTVmMmY3ZDFkZmUxMTZkMmYzZTg2YzlkZjYzZDhlZmEyOTgxNTk0MDZiMjZiYjliNjUxZTc0NjMyNywxLDE%3D --------------------------------------------------------- Développeur Fullstack PYTHON (IT) / Freelance Free-Work (ex Freelance-info Carriere-info) Paris et périphérie Voir l’offre d’emploi : https://www.linkedin.com/comm/jobs/view/3924434785/?trackingId=kpEzNtTJN2ZgODg5jYsFzQ%3D%3D&refId=ByteString%28length%3D16%2Cbytes%3Dee2a5148...a4daa673%29&lipi=urn%3Ali%3Apage%3Aemail_email_job_alert_digest_01%3Bvtw3tFEUQ%2BScN%2BCR5Nu6yg%3D%3D&midToken=AQErSc3Sdxmcdw&midSig=155djI3kRUlbg1&trk=eml-email_job_alert_digest_01-job_card-0-view_job&trkEmail=eml-email_job_alert_digest_01-job_card-0-view_job-null-bwd1d4~lw6uuuqc~7b-null-null&eid=bwd1d4-lw6uuuqc-7b&otpToken=MTUwNDE3ZTQxYTJkYzBjMGIwMjQwNGVkNDcxYmUwYjU4NmNjZDE0ODlhYTk4ZjYxNzljZjAzNmE0YzVkNTVmMmY3ZDFkZmUxMTZkMmYzZTg2YzlkZjYzZDhlZmEyOTgxNTk0MDZiMjZiYjliNjUxZTc0NjMyNywxLDE%3D --------------------------------------------------------- Voir toutes les offres d’emploi sur LinkedIn : https://www.linkedin.com/comm/jobs/search?geoId=105839861&f_TPR=a1715613301-&savedSearchId=1739556099&origin=JOB_ALERT_EMAIL&lipi=urn%3Ali%3Apage%3Aemail_email_job_alert_digest_01%3Bvtw3tFEUQ%2BScN%2BCR5Nu6yg%3D%3D&midToken=AQErSc3Sdxmcdw&midSig=155djI3kRUlbg1&trk=eml-email_job_alert_digest_01-job~alert-0-see~all~jobs~text&trkEmail=eml-email_job_alert_digest_01-job~alert-0-see~all~jobs~text-null-bwd1d4~lw6uuuqc~7b-null-null&eid=bwd1d4-lw6uuuqc-7b&otpToken=MTUwNDE3ZTQxYTJkYzBjMGIwMjQwNGVkNDcxYmUwYjU4NmNjZDE0ODlhYTk4ZjYxNzljZjAzNmE0YzVkNTVmMmY3ZDFkZmUxMTZkMmYzZTg2YzlkZjYzZDhlZmEyOTgxNTk0MDZiMjZiYjliNjUxZTc0NjMyNywxLDE%3D Découvrez les offres d’emploi pour lesquelles vous êtes l’un des meilleurs candidats https://www.linkedin.com/comm/premium/products/?utype=job&lipi=urn%3Ali%3Apage%3Aemail_email_job_alert_digest_01%3Bvtw3tFEUQ%2BScN%2BCR5Nu6yg%3D%3D&midToken=AQErSc3Sdxmcdw&midSig=155djI3kRUlbg1&trk=eml-email_job_alert_digest_01-job~alert-0-premium~upsell~text&trkEmail=eml-email_job_alert_digest_01-job~alert-0-premium~upsell~text-null-bwd1d4~lw6uuuqc~7b-null-null&eid=bwd1d4-lw6uuuqc-7b&otpToken=MTUwNDE3ZTQxYTJkYzBjMGIwMjQwNGVkNDcxYmUwYjU4NmNjZDE0ODlhYTk4ZjYxNzljZjAzNmE0YzVkNTVmMmY3ZDFkZmUxMTZkMmYzZTg2YzlkZjYzZDhlZmEyOTgxNTk0MDZiMjZiYjliNjUxZTc0NjMyNywxLDE%3D ---------------------------------------- Cet e-mail est destiné à Thibault Goutorbe (Développeur Python/R, spécialisé en Data/IA, à la recherche d’un emploi.) Découvrez pourquoi nous précisons ceci : https://www.linkedin.com/help/linkedin/answer/4788?lang=fr&lipi=urn%3Ali%3Apage%3Aemail_email_job_alert_digest_01%3Bvtw3tFEUQ%2BScN%2BCR5Nu6yg%3D%3D&midToken=AQErSc3Sdxmcdw&midSig=155djI3kRUlbg1&trk=eml-email_job_alert_digest_01-SecurityHelp-0-textfooterglimmer&trkEmail=eml-email_job_alert_digest_01-SecurityHelp-0-textfooterglimmer-null-bwd1d4~lw6uuuqc~7b-null-null&eid=bwd1d4-lw6uuuqc-7b&otpToken=MTUwNDE3ZTQxYTJkYzBjMGIwMjQwNGVkNDcxYmUwYjU4NmNjZDE0ODlhYTk4ZjYxNzljZjAzNmE0YzVkNTVmMmY3ZDFkZmUxMTZkMmYzZTg2YzlkZjYzZDhlZmEyOTgxNTk0MDZiMjZiYjliNjUxZTc0NjMyNywxLDE%3D Vous recevez des e-mails d’alertes Emploi. Gérez vos alertes Emploi : https://www.linkedin.com/comm/jobs/alerts?lipi=urn%3Ali%3Apage%3Aemail_email_job_alert_digest_01%3Bvtw3tFEUQ%2BScN%2BCR5Nu6yg%3D%3D&midToken=AQErSc3Sdxmcdw&midSig=155djI3kRUlbg1&trk=eml-email_job_alert_digest_01-null-0-null&trkEmail=eml-email_job_alert_digest_01-null-0-null-null-bwd1d4~lw6uuuqc~7b-null-null&eid=bwd1d4-lw6uuuqc-7b&otpToken=MTUwNDE3ZTQxYTJkYzBjMGIwMjQwNGVkNDcxYmUwYjU4NmNjZDE0ODlhYTk4ZjYxNzljZjAzNmE0YzVkNTVmMmY3ZDFkZmUxMTZkMmYzZTg2YzlkZjYzZDhlZmEyOTgxNTk0MDZiMjZiYjliNjUxZTc0NjMyNywxLDE%3D Se désabonner : https://www.linkedin.com/job-alert-email-unsubscribe?savedSearchId=1739556099&lipi=urn%3Ali%3Apage%3Aemail_email_job_alert_digest_01%3Bvtw3tFEUQ%2BScN%2BCR5Nu6yg%3D%3D&midToken=AQErSc3Sdxmcdw&midSig=155djI3kRUlbg1&ek=email_job_alert_digest_01&e=bwd1d4-lw6uuuqc-7b&eid=bwd1d4-lw6uuuqc-7b&m=unsubscribe&ts=footerGlimmer&li=0&t=plh · Aide : https://www.linkedin.com/help/linkedin/answer/67?lang=fr&lipi=urn%3Ali%3Apage%3Aemail_email_job_alert_digest_01%3Bvtw3tFEUQ%2BScN%2BCR5Nu6yg%3D%3D&midToken=AQErSc3Sdxmcdw&midSig=155djI3kRUlbg1&trk=eml-email_job_alert_digest_01-help-0-textfooterglimmer&trkEmail=eml-email_job_alert_digest_01-help-0-textfooterglimmer-null-bwd1d4~lw6uuuqc~7b-null-null&eid=bwd1d4-lw6uuuqc-7b&otpToken=MTUwNDE3ZTQxYTJkYzBjMGIwMjQwNGVkNDcxYmUwYjU4NmNjZDE0ODlhYTk4ZjYxNzljZjAzNmE0YzVkNTVmMmY3ZDFkZmUxMTZkMmYzZTg2YzlkZjYzZDhlZmEyOTgxNTk0MDZiMjZiYjliNjUxZTc0NjMyNywxLDE%3D © 2024 LinkedIn Ireland Unlimited Company, Wilton Plaza, Wilton Place, Dublin 2.LinkedIn est le nom commercial déposé de LinkedIn Ireland Unlimited Company. LinkedIn et le logo de LinkedIn sont des marques déposées de LinkedIn."
        },
        {
            "email_id": "daf0545f-65d8-40a6-80a4-b1277df98303",
            "from": "Zivan RASOLOFO <invitations@linkedin.com>",
            "subject": "Vous avez une invitation",
            "date": "Tue, 14 May 2024 20:40:15 +0000 (UTC)",
            "body": "Thibault Goutorbe, répondez à l’invitation de Zivan maintenant. Bonjour Thibault, je souhaiterais rejoindre votre réseau professionnel Zivan RASOLOFO Développeur en intelligence artificielle | Recherche alternance 16 mois - Début Septembre - Contrat de professionnalisation Paris et périphérie Accepter :https://www.linkedin.com/comm/mynetwork/invite-accept/invitationId/7196215498970202113/sharedKey/IiKNkw27/?inviterVanityName=zivan-r&lipi=urn%3Ali%3Apage%3Aemail_email_m2m_invite_single_01%3B%2FLCWR7KWQUyTxbhZIVj3XQ%3D%3D&midToken=AQErSc3Sdxmcdw&midSig=155djI3kRUlbg1&trk=eml-email_m2m_invite_single_01-null-0-null&trkEmail=eml-email_m2m_invite_single_01-null-0-null-null-bwd1d4~lw6uuuqc~nd-null-null&eid=bwd1d4-lw6uuuqc-nd&otpToken=MTUwNDE3ZTQxYTJkYzBjMGIwMjQwNGVkNDcxYmUwYjU4NmNjZDE0NTllYWQ4ZDYxNzljZjAzNmE0YzVkNTVmMmY3ZDFkZmUxNzFlN2NmYzI0ZWJmZjFiZTE1MDJhOWQxN2MwYTk5MzcxM2FjOGY1NDUyMDdiYiwxLDE%3D Voir le profil :https://fr.linkedin.com/comm/in/zivan-r?lipi=urn%3Ali%3Apage%3Aemail_email_m2m_invite_single_01%3B%2FLCWR7KWQUyTxbhZIVj3XQ%3D%3D&midToken=AQErSc3Sdxmcdw&midSig=155djI3kRUlbg1&trk=eml-email_m2m_invite_single_01-null-0-null&trkEmail=eml-email_m2m_invite_single_01-null-0-null-null-bwd1d4~lw6uuuqc~nd-null-null&eid=bwd1d4-lw6uuuqc-nd&otpToken=MTUwNDE3ZTQxYTJkYzBjMGIwMjQwNGVkNDcxYmUwYjU4NmNjZDE0NTllYWQ4ZDYxNzljZjAzNmE0YzVkNTVmMmY3ZDFkZmUxNzFlN2NmYzI0ZWJmZjFiZTE1MDJhOWQxN2MwYTk5MzcxM2FjOGY1NDUyMDdiYiwxLDE%3D ---------------------------------------- Cet e-mail est destiné à Thibault Goutorbe (Développeur Python/R, spécialisé en Data/IA, à la recherche d’un emploi.) Découvrez pourquoi nous précisons ceci : https://www.linkedin.com/help/linkedin/answer/4788?lang=fr&lipi=urn%3Ali%3Apage%3Aemail_email_m2m_invite_single_01%3B%2FLCWR7KWQUyTxbhZIVj3XQ%3D%3D&midToken=AQErSc3Sdxmcdw&midSig=155djI3kRUlbg1&trk=eml-email_m2m_invite_single_01-SecurityHelp-0-textfooterglimmer&trkEmail=eml-email_m2m_invite_single_01-SecurityHelp-0-textfooterglimmer-null-bwd1d4~lw6uuuqc~nd-null-null&eid=bwd1d4-lw6uuuqc-nd&otpToken=MTUwNDE3ZTQxYTJkYzBjMGIwMjQwNGVkNDcxYmUwYjU4NmNjZDE0NTllYWQ4ZDYxNzljZjAzNmE0YzVkNTVmMmY3ZDFkZmUxNzFlN2NmYzI0ZWJmZjFiZTE1MDJhOWQxN2MwYTk5MzcxM2FjOGY1NDUyMDdiYiwxLDE%3D Vous recevez des invitations LinkedIn par e-mail. Se désabonner : https://www.linkedin.com/comm/psettings/email-unsubscribe?lipi=urn%3Ali%3Apage%3Aemail_email_m2m_invite_single_01%3B%2FLCWR7KWQUyTxbhZIVj3XQ%3D%3D&midToken=AQErSc3Sdxmcdw&midSig=155djI3kRUlbg1&trk=eml-email_m2m_invite_single_01-unsubscribe-0-textfooterglimmer&trkEmail=eml-email_m2m_invite_single_01-unsubscribe-0-textfooterglimmer-null-bwd1d4~lw6uuuqc~nd-null-null&eid=bwd1d4-lw6uuuqc-nd&loid=AQG89yb0wzbL5gAAAY941ZZ65l4znfS0hxJWt156vl6X0TH65Sd15LbzMOR5zYmWAQEJw5eHBXVkium1zmVTVXZxNSUIVI_pvwGHCso Aide : https://www.linkedin.com/help/linkedin/answer/67?lang=fr&lipi=urn%3Ali%3Apage%3Aemail_email_m2m_invite_single_01%3B%2FLCWR7KWQUyTxbhZIVj3XQ%3D%3D&midToken=AQErSc3Sdxmcdw&midSig=155djI3kRUlbg1&trk=eml-email_m2m_invite_single_01-help-0-textfooterglimmer&trkEmail=eml-email_m2m_invite_single_01-help-0-textfooterglimmer-null-bwd1d4~lw6uuuqc~nd-null-null&eid=bwd1d4-lw6uuuqc-nd&otpToken=MTUwNDE3ZTQxYTJkYzBjMGIwMjQwNGVkNDcxYmUwYjU4NmNjZDE0NTllYWQ4ZDYxNzljZjAzNmE0YzVkNTVmMmY3ZDFkZmUxNzFlN2NmYzI0ZWJmZjFiZTE1MDJhOWQxN2MwYTk5MzcxM2FjOGY1NDUyMDdiYiwxLDE%3D © 2024 LinkedIn Ireland Unlimited Company, Wilton Plaza, Wilton Place, Dublin 2.LinkedIn est le nom commercial déposé de LinkedIn Ireland Unlimited Company. LinkedIn et le logo de LinkedIn sont des marques déposées de LinkedIn."
        }
    ]
}

What fake mails generate by chatgpt look like:

In [ ]:
{
    "emails": [
        {
            "id": 1,
            "from": "etudiant1@example.com",
            "subject": "Question sur les thèmes de Jane Austen",
            "date": "2023-10-01",
            "body": "Bonjour, je voulais vous poser une question concernant les thèmes principaux dans les œuvres de Jane Austen, notamment dans \"Pride and Prejudice\". Pouvez-vous m'expliquer comment les relations sociales sont dépeintes ? Merci beaucoup !"
        },
        {
            "id": 2,
            "from": "etudiant2@example.com",
            "subject": "Problème avec les intégrales",
            "date": "2023-10-02",
            "body": "Bonjour, j'ai des difficultés avec le calcul des intégrales. Pourriez-vous m'expliquer comment on résout les intégrales définies, en particulier celles qui impliquent des fonctions exponentielles ? Merci !"
        },
        {
            "id": 3,
            "from": "etudiant3@example.com",
            "subject": "Compréhension de la régression linéaire",
            "date": "2023-10-03",
            "body": "Bonjour, j'étudie la régression linéaire et j'ai du mal à comprendre comment interpréter les coefficients de régression. Pourriez-vous m'expliquer comment ils influencent le modèle prédictif ? Merci d'avance."
        },
        {
            "id": 4,
            "from": "etudiant4@example.com",
            "subject": "Demande d'éclaircissement sur les poèmes de William Wordsworth",
            "date": "2023-10-04",
            "body": "Bonjour, j'ai besoin de quelques éclaircissements sur le style de William Wordsworth, en particulier sur sa représentation de la nature. Auriez-vous des conseils pour mieux interpréter ses poèmes ? Merci pour votre aide !"
        },
        {
            "id": 5,
            "from": "etudiant5@example.com",
            "subject": "Calcul des dérivées secondes",
            "date": "2023-10-05",
            "body": "Bonjour, je suis un peu perdu avec les dérivées secondes, surtout dans les cas où la fonction est une somme de plusieurs termes. Pourriez-vous m'aider à comprendre cela ? Merci !"
        },
    ]
}

What the part that will sens message is looking like for now (just a prototype)

In [ ]:
import os.path
import base64
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import json

# Scopes pour lire et envoyer des emails
SCOPES = ['https://www.googleapis.com/auth/gmail.send']


# Fonction pour créer le message MIME
def create_message(sender, to, subject, message_text):
    message = MIMEMultipart()
    message['From'] = sender
    message['To'] = to
    message['Subject'] = subject
    message.attach(MIMEText(message_text, 'plain'))
    raw_message = base64.urlsafe_b64encode(message.as_bytes()).decode()
    return {'raw': raw_message}

# Fonction pour envoyer un email via l'API Gmail
def send_email(service, sender, to, subject, message_text):
    message = create_message(sender, to, subject, message_text)
    try:
        sent_message = service.users().messages().send(userId="me", body=message).execute()
        print(f"Message sent to {to}, Message Id: {sent_message['id']}")
    except Exception as e:
        print(f"An error occurred: {e}")

# Charger les destinataires depuis un fichier JSON
def load_recipients(filename):
    try:
        with open(filename, 'r', encoding='utf-8') as file:
            data = json.load(file)
            return data['recipients']
    except Exception as e:
        print(f"Error loading recipients: {e}")
        return []

# Envoi des emails à partir de la liste des destinataires
def send_emails_to_recipients(service, sender, filename, subject, body):
    recipients = load_recipients(filename)
    for recipient in recipients:
        email_address = recipient.get("email")
        if email_address:
            send_email(service, sender, email_address, subject, body)
        else:
            print(f"No valid email address for recipient: {recipient}")

# Utilisation du script pour envoyer un email
if __name__ == "__main__":
    # Authentifier l'utilisateur et créer le service Gmail
    service = authenticate_gmail()

    # Définir l'expéditeur, le sujet et le corps du message
    sender_email = "your_email@gmail.com"
    subject = "Notification"
    body = "This is a test email sent from the automated system."

    # Envoyer les emails
    send_emails_to_recipients(service, sender_email, "codes/addresses.json", subject, body)


What outputs look like after the prediction:

Documents\AIVANCITY\ML Business> & C:/Users/VivoBook/anaconda3/envs/Python3125/python.exe "c:/Users/VivoBook/Documents/AIVANCITY/ML Business/codes/testrecognition.py"  
```python
{
    "mots_clés": ["thèmes de Jane Austen", "Pride and Prejudice", "relations sociales"],
    "prédiction_personne": ["professeur de littérature", "spécialiste de Jane Austen"]
}
```
{
    "mots_clés": ["intégrales", "calcul", "intégrales définies", "fonctions exponentielles"],
    "personne": ["professeur de mathématiques"]
}
```python
{
    "mots_clés": ["régression linéaire", "coefficients de régression", "modèle prédictif"],
    "prédiction_personne": ["professeur de statistiques", "enseignant en mathématiques", "tuteur en analyse de données"]
}
```
```python
{
    "mots_clés": ["William Wordsworth", "poèmes", "style", "nature", "interpréter"],
    "prédiction_personne": ["professeur de littérature", "spécialiste en littérature", "critique littéraire"]
}
```
```python
{
    "mots_clés": ["dérivées secondes", "fonction", "somme de plusieurs termes"],
    "prédiction_personne": ["professeur de mathématiques", "tuteur de mathématiques"]
}
```
```python
{
    "mots_clés": ["réseaux de neurones", "rétropropagation", "gradients"],
    "personne": ["Responsable IA", "Data Scientist", "Ingénieur en Machine Learning"]
}
```
```python
{
    "mots_clés": ["Hamlet", "Shakespeare", "monologue", "To be or not to be", "analyse"],
    "prédiction_personne": ["Professeur de littérature", "Spécialiste de Shakespeare", "Enseignant en littérature"]
}
```
```python
{
    "mots_clés": ["suites arithmétiques", "suites géométriques", "théorèmes", "démonstration"],
    "prédiction_personne": ["Professeur de mathématiques", "Tuteur en mathématiques"]
}
```
```python
{
    "mots_clés": ["apprentissage par renforcement", "récompense", "comportement des modèles"],
    "prédiction_personne": ["Professeur d'intelligence artificielle", "Spécialiste en apprentissage par renforcement", "Ingénieur en machine learning"]
}
```
PS C:\Users\VivoBook\Documents\AIVANCITY\ML Business> 